## 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 임포트

In [2]:
# 필요한 라이브러리 import
import os
import json
import datetime
import shutil
import time
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np

## EDA

In [3]:
class Config:
    drive_root = '/content/drive/MyDrive/의장공정/데이터/Validation'
    original_data_path = os.path.join(drive_root, '원천데이터')
    label_data_path = os.path.join(drive_root, '라벨링데이터')

def load_and_analyze_data():
    """원천데이터와 라벨링데이터를 로드하고 분석합니다."""
    print("데이터 로딩 및 분석 중...")

    # 원천데이터 파일 목록
    original_files = []
    for root, dirs, files in os.walk(Config.original_data_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                original_files.append(os.path.join(root, file))

    # 라벨링데이터 파일 목록
    label_files = []
    for root, dirs, files in os.walk(Config.label_data_path):
        for file in files:
            if file.lower().endswith('.json'):
                label_files.append(os.path.join(root, file))

    print(f"원천데이터 이미지 수: {len(original_files)}")
    print(f"라벨링데이터 파일 수: {len(label_files)}")

    return original_files, label_files

def analyze_label_structure(label_files, num_samples=5):
    """라벨링 데이터의 구조를 분석합니다."""
    print(f"라벨링 데이터 구조 분석 중... (샘플 {num_samples}개)")

    for i, label_file in enumerate(label_files[:num_samples]):
        print(f"\n--- 샘플 {i+1}: {os.path.basename(label_file)} ---")
        try:
            with open(label_file, 'r', encoding='utf-8-sig') as f:
                label_data = json.load(f)

            print("JSON 구조:")
            print(json.dumps(label_data, ensure_ascii=False, indent=2)[:500] + "...")

            # 주요 키들 확인
            print(f"주요 키들: {list(label_data.keys())}")

        except Exception as e:
            print(f"오류: {e}")

    print("\n" + "="*50)

def analyze_class_distribution(paired_data):
    """클래스별 데이터 개수 분석"""
    class_counts = {}
    category_counts = {}
    quality_counts = {}

    for item in paired_data:
        label = item['label']
        category = item.get('category_name', 'unknown')
        quality = item.get('quality', 'unknown')

        class_counts[label] = class_counts.get(label, 0) + 1
        category_counts[category] = category_counts.get(category, 0) + 1
        quality_counts[quality] = quality_counts.get(quality, 0) + 1

    print("\n=== 클래스별 데이터 개수 ===")
    for label, count in sorted(class_counts.items()):
        print(f"  {label}: {count}개")

    print("\n=== 카테고리별 데이터 개수 ===")
    for category, count in sorted(category_counts.items()):
        print(f"  {category}: {count}개")

    print("\n=== 품질별 데이터 개수 ===")
    for quality, count in sorted(quality_counts.items()):
        print(f"  {quality}: {count}개")

def analyze_file_structure():
    """현재 복사된 파일 개수 확인"""
    dataset_dir = '/content/drive/MyDrive/의장공정/데이터/image_classification/dataset'
    splits = ['train', 'valid', 'test']

    print("=== 현재 복사된 파일 개수 확인 ===")
    total_files = 0

    for split in splits:
        split_dir = os.path.join(dataset_dir, split)

        if os.path.exists(split_dir):
            files = [f for f in os.listdir(split_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            file_count = len(files)
            total_files += file_count

            print(f"{split}: {file_count:,}개")

            # 첫 3개 파일명 확인
            if files:
                print(f"  예시 파일명:")
                for i, file in enumerate(files[:3]):
                    print(f"    {file}")
            print()
        else:
            print(f"{split}: 폴더 없음")

    print(f"전체 복사된 파일: {total_files:,}개")
    return total_files

def create_image_label_pairs(original_files, label_files):
    """이미지와 라벨 파일을 매칭하여 데이터셋을 생성합니다."""
    print("이미지-라벨 매칭 중...")

    # 먼저 라벨 데이터 구조 분석
    analyze_label_structure(label_files)

    # 1000개씩 나누어서 처리
    batch_size = 1000
    total_files = len(original_files)
    all_paired_data = []
    all_labels_set = set()

    # 라벨 딕셔너리는 미리 생성
    label_dict = {}
    for label_file in label_files:
        base_name = os.path.splitext(os.path.basename(label_file))[0]
        label_dict[base_name] = label_file

    print(f"라벨 딕셔너리 생성 완료: {len(label_dict)}개")

    # 배치별로 처리
    for start_idx in range(0, total_files, batch_size):
        end_idx = min(start_idx + batch_size, total_files)
        batch_files = original_files[start_idx:end_idx]

        print(f"배치 처리 중: {start_idx}-{end_idx} ({start_idx/total_files*100:.1f}%)")

        # 이 배치만 처리
        batch_paired_data = []
        for img_file in batch_files:
            img_base_name = os.path.splitext(os.path.basename(img_file))[0]

            if img_base_name in label_dict:
                try:
                    with open(label_dict[img_base_name], 'r', encoding='utf-8-sig') as f:
                        label_data = json.load(f)

                    # JSON 구조에 맞는 라벨 추출
                    label = None
                    category_name = None
                    quality = None

                    # categories 딕셔너리 생성
                    categories_dict = {}
                    if 'categories' in label_data:
                        for cat in label_data['categories']:
                            categories_dict[cat['id']] = cat['name']

                    # annotations에서 정보 추출
                    if 'annotations' in label_data and len(label_data['annotations']) > 0:
                        annotation = label_data['annotations'][0]

                        if 'category_id' in annotation and annotation['category_id'] in categories_dict:
                            category_name = categories_dict[annotation['category_id']]

                        if 'attributes' in annotation and 'quality' in annotation['attributes']:
                            quality = annotation['attributes']['quality']

                    # 라벨 생성
                    if category_name and quality:
                        label = f"{category_name}_{quality}"
                    elif category_name:
                        label = category_name
                    else:
                        # 폴더 경로에서 라벨 추출
                        folder_parts = img_file.split(os.sep)
                        if len(folder_parts) >= 3:
                            label = f"{folder_parts[-3]}_{folder_parts[-2]}"
                        else:
                            label = 'unknown'

                    batch_paired_data.append({
                        'image_path': img_file,
                        'label_path': label_dict[img_base_name],
                        'label': str(label),
                        'image_name': os.path.basename(img_file),
                        'category_name': category_name,
                        'quality': quality
                    })
                    all_labels_set.add(str(label))

                except Exception as e:
                    print(f"  오류 건너뜀: {e}")

        all_paired_data.extend(batch_paired_data)
        print(f"  이 배치 결과: {len(batch_paired_data)}개 매칭")

        # 메모리 정리
        del batch_paired_data

    print(f"\n매칭 완료!")
    print(f"매칭된 이미지-라벨 쌍: {len(all_paired_data)}")
    print(f"발견된 클래스 수: {len(all_labels_set)}")

    # 클래스별 분포 분석
    analyze_class_distribution(all_paired_data)

    return all_paired_data, sorted(list(all_labels_set))

# 실행 함수
def run_eda():
    """전체 EDA 실행"""
    print("=== 이미지 분류 EDA 시작 ===")

    # 1. 데이터 로드 및 분석
    original_files, label_files = load_and_analyze_data()

    # 2. 이미지-라벨 매칭
    paired_data, class_names = create_image_label_pairs(original_files, label_files)

    # 3. 파일 구조 분석
    analyze_file_structure()

    print("\n=== EDA 완료 ===")
    return paired_data, class_names

# 실행
if __name__ == "__main__":
    paired_data, class_names = run_eda()

=== 이미지 분류 EDA 시작 ===
데이터 로딩 및 분석 중...


KeyboardInterrupt: 

In [ ]:
# 이미지 분류 EDA (탐색적 데이터 분석) 코드

import os
import json
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

class Config:
    drive_root = '/content/drive/MyDrive/의장공정/데이터/Validation'
    original_data_path = os.path.join(drive_root, '원천데이터')
    label_data_path = os.path.join(drive_root, '라벨링데이터')

def load_and_analyze_data():
    """원천데이터와 라벨링데이터를 로드하고 분석합니다."""
    print("데이터 로딩 및 분석 중...")

    # 원천데이터 파일 목록
    original_files = []
    for root, dirs, files in os.walk(Config.original_data_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                original_files.append(os.path.join(root, file))

    # 라벨링데이터 파일 목록
    label_files = []
    for root, dirs, files in os.walk(Config.label_data_path):
        for file in files:
            if file.lower().endswith('.json'):
                label_files.append(os.path.join(root, file))

    print(f"원천데이터 이미지 수: {len(original_files)}")
    print(f"라벨링데이터 파일 수: {len(label_files)}")

    return original_files, label_files

def analyze_label_structure(label_files, num_samples=5):
    """라벨링 데이터의 구조를 분석합니다."""
    print(f"라벨링 데이터 구조 분석 중... (샘플 {num_samples}개)")

    for i, label_file in enumerate(label_files[:num_samples]):
        print(f"\n--- 샘플 {i+1}: {os.path.basename(label_file)} ---")
        try:
            with open(label_file, 'r', encoding='utf-8-sig') as f:
                label_data = json.load(f)

            print("JSON 구조:")
            print(json.dumps(label_data, ensure_ascii=False, indent=2)[:500] + "...")

            # 주요 키들 확인
            print(f"주요 키들: {list(label_data.keys())}")

        except Exception as e:
            print(f"오류: {e}")

    print("\n" + "="*50)

def analyze_class_distribution(paired_data):
    """클래스별 데이터 개수 분석"""
    class_counts = {}
    category_counts = {}
    quality_counts = {}

    for item in paired_data:
        label = item['label']
        category = item.get('category_name', 'unknown')
        quality = item.get('quality', 'unknown')

        class_counts[label] = class_counts.get(label, 0) + 1
        category_counts[category] = category_counts.get(category, 0) + 1
        quality_counts[quality] = quality_counts.get(quality, 0) + 1

    print("\n=== 클래스별 데이터 개수 ===")
    for label, count in sorted(class_counts.items()):
        print(f"  {label}: {count}개")

    print("\n=== 카테고리별 데이터 개수 ===")
    for category, count in sorted(category_counts.items()):
        print(f"  {category}: {count}개")

    print("\n=== 품질별 데이터 개수 ===")
    for quality, count in sorted(quality_counts.items()):
        print(f"  {quality}: {count}개")

def analyze_file_structure():
    """현재 복사된 파일 개수 확인"""
    dataset_dir = '/content/drive/MyDrive/의장공정/데이터/image_classification/dataset'
    splits = ['train', 'valid', 'test']

    print("=== 현재 복사된 파일 개수 확인 ===")
    total_files = 0

    for split in splits:
        split_dir = os.path.join(dataset_dir, split)

        if os.path.exists(split_dir):
            files = [f for f in os.listdir(split_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            file_count = len(files)
            total_files += file_count

            print(f"{split}: {file_count:,}개")

            # 첫 3개 파일명 확인
            if files:
                print(f"  예시 파일명:")
                for i, file in enumerate(files[:3]):
                    print(f"    {file}")
            print()
        else:
            print(f"{split}: 폴더 없음")

    print(f"전체 복사된 파일: {total_files:,}개")
    return total_files

def create_image_label_pairs(original_files, label_files):
    """이미지와 라벨 파일을 매칭하여 데이터셋을 생성합니다."""
    print("이미지-라벨 매칭 중...")

    # 먼저 라벨 데이터 구조 분석
    analyze_label_structure(label_files)

    # 1000개씩 나누어서 처리
    batch_size = 1000
    total_files = len(original_files)
    all_paired_data = []
    all_labels_set = set()

    # 라벨 딕셔너리는 미리 생성
    label_dict = {}
    for label_file in label_files:
        base_name = os.path.splitext(os.path.basename(label_file))[0]
        label_dict[base_name] = label_file

    print(f"라벨 딕셔너리 생성 완료: {len(label_dict)}개")

    # 배치별로 처리
    for start_idx in range(0, total_files, batch_size):
        end_idx = min(start_idx + batch_size, total_files)
        batch_files = original_files[start_idx:end_idx]

        print(f"배치 처리 중: {start_idx}-{end_idx} ({start_idx/total_files*100:.1f}%)")

        # 이 배치만 처리
        batch_paired_data = []
        for img_file in batch_files:
            img_base_name = os.path.splitext(os.path.basename(img_file))[0]

            if img_base_name in label_dict:
                try:
                    with open(label_dict[img_base_name], 'r', encoding='utf-8-sig') as f:
                        label_data = json.load(f)

                    # JSON 구조에 맞는 라벨 추출
                    label = None
                    category_name = None
                    quality = None

                    # categories 딕셔너리 생성
                    categories_dict = {}
                    if 'categories' in label_data:
                        for cat in label_data['categories']:
                            categories_dict[cat['id']] = cat['name']

                    # annotations에서 정보 추출
                    if 'annotations' in label_data and len(label_data['annotations']) > 0:
                        annotation = label_data['annotations'][0]

                        if 'category_id' in annotation and annotation['category_id'] in categories_dict:
                            category_name = categories_dict[annotation['category_id']]

                        if 'attributes' in annotation and 'quality' in annotation['attributes']:
                            quality = annotation['attributes']['quality']

                    # 라벨 생성
                    if category_name and quality:
                        label = f"{category_name}_{quality}"
                    elif category_name:
                        label = category_name
                    else:
                        # 폴더 경로에서 라벨 추출
                        folder_parts = img_file.split(os.sep)
                        if len(folder_parts) >= 3:
                            label = f"{folder_parts[-3]}_{folder_parts[-2]}"
                        else:
                            label = 'unknown'

                    batch_paired_data.append({
                        'image_path': img_file,
                        'label_path': label_dict[img_base_name],
                        'label': str(label),
                        'image_name': os.path.basename(img_file),
                        'category_name': category_name,
                        'quality': quality
                    })
                    all_labels_set.add(str(label))

                except Exception as e:
                    print(f"  오류 건너뜀: {e}")

        all_paired_data.extend(batch_paired_data)
        print(f"  이 배치 결과: {len(batch_paired_data)}개 매칭")

        # 메모리 정리
        del batch_paired_data

    print(f"\n매칭 완료!")
    print(f"매칭된 이미지-라벨 쌍: {len(all_paired_data)}")
    print(f"발견된 클래스 수: {len(all_labels_set)}")

    # 클래스별 분포 분석
    analyze_class_distribution(all_paired_data)

    return all_paired_data, sorted(list(all_labels_set))

# 실행 함수
def run_eda():
    """전체 EDA 실행"""
    print("=== 이미지 분류 EDA 시작 ===")

    # 1. 데이터 로드 및 분석
    original_files, label_files = load_and_analyze_data()

    # 2. 이미지-라벨 매칭
    paired_data, class_names = create_image_label_pairs(original_files, label_files)

    # 3. 파일 구조 분석
    analyze_file_structure()

    print("\n=== EDA 완료 ===")
    return paired_data, class_names

# 실행
if __name__ == "__main__":
    paired_data, class_names = run_eda()

### EDA 결과

In [4]:
# 데이터 로딩 및 분석 중...
# 원천데이터 이미지 수: 21553
# 라벨링데이터 파일 수: 21553
# 이미지-라벨 매칭 중...
# 라벨링 데이터 구조 분석 중... (샘플 5개)

# --- 샘플 1: 204_101_20_1bace470-0fad-4b08-a0f2-f399063a9f6f.json ---
# JSON 구조:
# {
#   "info": {
#     "contributor": "미래아이티컨소시엄",
#     "date_created": "2021-12-02 08:54:24.241591",
#     "name": "부품 품질 검사 영상 데이터(자동차)",
#     "description": "부품 품질 검사 영상 데이터(자동차)",
#     "version": "1.0",
#     "url": "miraeit.net/"
#   },
#   "images": [
#     {
#       "license": 1,
#       "file_name": "204_101_20_1bace470-0fad-4b08-a0f2-f399063a9f6f.jpg",
#       "width": 4000,
#       "date_captured": "2021-11-25 13:57:21",
#       "id": 1,
#       "height": 3000
#     }
#   ],
#   "licenses": [
#     {
#       "name": "CC BY-N...
# 주요 키들: ['info', 'images', 'licenses', 'categories', 'annotations']

# --- 샘플 2: 204_101_20_1bf6a4a7-ef91-4efb-aee4-965af8326c49.json ---
# JSON 구조:
# {
#   "info": {
#     "contributor": "미래아이티컨소시엄",
#     "date_created": "2021-12-24 13:52:22.338968",
#     "name": "부품 품질 검사 영상 데이터(자동차)",
#     "description": "부품 품질 검사 영상 데이터(자동차)",
#     "version": "1.0",
#     "url": "miraeit.net/"
#   },
#   "images": [
#     {
#       "license": 1,
#       "file_name": "204_101_20_1bf6a4a7-ef91-4efb-aee4-965af8326c49.JPG",
#       "width": 4032,
#       "date_captured": "2021-12-15 21:23:29",
#       "id": 1,
#       "height": 2268
#     }
#   ],
#   "licenses": [
#     {
#       "name": "CC BY-N...
# 주요 키들: ['info', 'images', 'licenses', 'categories', 'annotations']

# --- 샘플 3: 204_101_20_1c65599f-3bc1-4297-9179-fc6f5fb8c0ea.json ---
# JSON 구조:
# {
#   "info": {
#     "contributor": "미래아이티컨소시엄",
#     "date_created": "2021-12-24 15:34:33.045516",
#     "name": "부품 품질 검사 영상 데이터(자동차)",
#     "description": "부품 품질 검사 영상 데이터(자동차)",
#     "version": "1.0",
#     "url": "miraeit.net/"
#   },
#   "images": [
#     {
#       "license": 1,
#       "file_name": "204_101_20_1c65599f-3bc1-4297-9179-fc6f5fb8c0ea.jpg",
#       "width": 1920,
#       "date_captured": "2021-12-14 12:12:41",
#       "id": 1,
#       "height": 1080
#     }
#   ],
#   "licenses": [
#     {
#       "name": "CC BY-N...
# 주요 키들: ['info', 'images', 'licenses', 'categories', 'annotations']

# --- 샘플 4: 204_101_20_1cd1e82c-0736-4442-b377-aafd5091528c.json ---
# JSON 구조:
# {
#   "info": {
#     "contributor": "미래아이티컨소시엄",
#     "date_created": "2021-12-20 15:54:00.28187",
#     "name": "부품 품질 검사 영상 데이터(자동차)",
#     "description": "부품 품질 검사 영상 데이터(자동차)",
#     "version": "1.0",
#     "url": "miraeit.net/"
#   },
#   "images": [
#     {
#       "license": 1,
#       "file_name": "204_101_20_1cd1e82c-0736-4442-b377-aafd5091528c.jpg",
#       "width": 4224,
#       "date_captured": "2021-12-14 11:43:57",
#       "id": 1,
#       "height": 2376
#     }
#   ],
#   "licenses": [
#     {
#       "name": "CC BY-NC...
# 주요 키들: ['info', 'images', 'licenses', 'categories', 'annotations']

# --- 샘플 5: 204_101_20_1cf0b04f-72b3-4b7d-8702-14eddef8996b.json ---
# JSON 구조:
# {
#   "info": {
#     "contributor": "미래아이티컨소시엄",
#     "date_created": "2021-12-25 16:48:45.309279",
#     "name": "부품 품질 검사 영상 데이터(자동차)",
#     "description": "부품 품질 검사 영상 데이터(자동차)",
#     "version": "1.0",
#     "url": "miraeit.net/"
#   },
#   "images": [
#     {
#       "license": 1,
#       "file_name": "204_101_20_1cf0b04f-72b3-4b7d-8702-14eddef8996b.jpg",
#       "width": 4624,
#       "date_captured": "2021-12-16 14:56:20",
#       "id": 1,
#       "height": 2084
#     }
#   ],
#   "licenses": [
#     {
#       "name": "CC BY-N...
# 주요 키들: ['info', 'images', 'licenses', 'categories', 'annotations']

# ==================================================
# 라벨 딕셔너리 생성 완료: 21553개
# 배치 처리 중: 0-1000 (0.0%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 1000-2000 (4.6%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 2000-3000 (9.3%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 3000-4000 (13.9%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 4000-5000 (18.6%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 5000-6000 (23.2%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 6000-7000 (27.8%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 7000-8000 (32.5%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 8000-9000 (37.1%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 9000-10000 (41.8%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 10000-11000 (46.4%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 11000-12000 (51.0%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 12000-13000 (55.7%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 13000-14000 (60.3%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 14000-15000 (65.0%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 15000-16000 (69.6%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 16000-17000 (74.2%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 17000-18000 (78.9%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 18000-19000 (83.5%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 19000-20000 (88.2%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 20000-21000 (92.8%)
#   이 배치 결과: 1000개 매칭
# 배치 처리 중: 21000-21553 (97.4%)
#   이 배치 결과: 553개 매칭

# 매칭 완료!
# 매칭된 이미지-라벨 쌍: 21553
# 발견된 클래스 수: 24

# === 클래스별 데이터 개수 ===
#   고정 불량_불량품: 757개
#   고정 불량_양품: 603개
#   고정핀 불량_불량품: 651개
#   고정핀 불량_양품: 644개
#   단차_불량품: 2714개
#   단차_양품: 2718개
#   스크래치_불량품: 1752개
#   스크래치_양품: 895개
#   실링 불량_불량품: 423개
#   실링 불량_양품: 214개
#   연계 불량_불량품: 636개
#   연계 불량_양품: 618개
#   외관 손상_불량품: 1152개
#   외관 손상_양품: 1060개
#   유격 불량_불량품: 982개
#   유격 불량_양품: 630개
#   장착 불량_불량품: 645개
#   장착 불량_양품: 650개
#   체결 불량_불량품: 1337개
#   체결 불량_양품: 587개
#   헤밍 불량_불량품: 543개
#   헤밍 불량_양품: 506개
#   홀 변형_불량품: 401개
#   홀 변형_양품: 435개

# === 카테고리별 데이터 개수 ===
#   고정 불량: 1360개
#   고정핀 불량: 1295개
#   단차: 5432개
#   스크래치: 2647개
#   실링 불량: 637개
#   연계 불량: 1254개
#   외관 손상: 2212개
#   유격 불량: 1612개
#   장착 불량: 1295개
#   체결 불량: 1924개
#   헤밍 불량: 1049개
#   홀 변형: 836개

# === 품질별 데이터 개수 ===
#   불량품: 11993개
#   양품: 9560개
# 데이터셋 분할 및 복사 중...
# 클래스 고정 불량_불량품: train=529, valid=114, test=114
# 클래스 고정 불량_양품: train=422, valid=90, test=91
# 클래스 고정핀 불량_불량품: train=455, valid=98, test=98
# 클래스 고정핀 불량_양품: train=450, valid=97, test=97
# 클래스 단차_불량품: train=1899, valid=407, test=408
# 클래스 단차_양품: train=1902, valid=408, test=408
# 클래스 스크래치_불량품: train=1226, valid=263, test=263
# 클래스 스크래치_양품: train=626, valid=134, test=135
# 클래스 실링 불량_불량품: train=296, valid=63, test=64
# 클래스 실링 불량_양품: train=149, valid=32, test=33
# 클래스 연계 불량_불량품: train=445, valid=95, test=96
# 클래스 연계 불량_양품: train=432, valid=93, test=93
# 클래스 외관 손상_불량품: train=806, valid=173, test=173
# 클래스 외관 손상_양품: train=742, valid=159, test=159
# 클래스 유격 불량_불량품: train=687, valid=147, test=148
# 클래스 유격 불량_양품: train=441, valid=94, test=95
# 클래스 장착 불량_불량품: train=451, valid=97, test=97
# 클래스 장착 불량_양품: train=455, valid=97, test=98
# 클래스 체결 불량_불량품: train=935, valid=201, test=201
# 클래스 체결 불량_양품: train=410, valid=88, test=89
# 클래스 헤밍 불량_불량품: train=380, valid=81, test=82
# 클래스 헤밍 불량_양품: train=354, valid=76, test=76
# 클래스 홀 변형_불량품: train=280, valid=60, test=61
# 클래스 홀 변형_양품: train=304, valid=65, test=66

# train 데이터 복사 중... (15076개)

# valid 데이터 복사 중... (3232개)

# test 데이터 복사 중... (3245개)
# 훈련 데이터가 없습니다. 종료합니다.